<a href="https://colab.research.google.com/github/adity1234567/Deep-Dive-in-The-Field-of-Misinformation/blob/main/Deep_Learning_For_NLP_Zero_To_Transformers_%26_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/code/tanulsingh077/deep-learning-for-nlp-zero-to-transformers-bert

Contents

In this Notebook I will start with the very Basics of RNN's and Build all the way to latest deep learning architectures to solve NLP problems. It will cover the Following:

- Simple RNN's

- Word Embeddings : Definition and How to get them
- LSTM's
- GRU's
- BI-Directional RNN's
- Encoder-Decoder Models (Seq2Seq Models)
- Attention Models
- Transformers - Attention is all you need
- BERT

In [1]:
import plotly.graph_objs as go  # for advanced customization of figures
import plotly.express as px     # for easier plotting
import plotly.figure_factory as ff  # for specialized charts


In [2]:
pip install tensorflow --upgrade


In [3]:
pip install numpy pandas scikit-learn tensorflow keras matplotlib seaborn plotly


In [6]:
import numpy as np


import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, GRU, SimpleRNN, Dense, Activation, Dropout, Embedding, BatchNormalization, GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from tensorflow.keras.preprocessing import sequence, text

from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff


Configuring TPU's

For this version of Notebook we will be using TPU's as we have to built a BERT Model **bold text**

In [7]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [11]:
!unzip '/content/drive/MyDrive/Health misinformation/zero to ransfer bert/jigsaw-toxic-comment-train.csv.zip' -d '/content/drive/MyDrive/Health misinformation/zero to ransfer bert/'
!unzip '/content/drive/MyDrive/Health misinformation/zero to ransfer bert/validation.csv.zip' -d '/content/drive/MyDrive/Health misinformation/zero to ransfer bert/'
!unzip '/content/drive/MyDrive/Health misinformation/zero to ransfer bert/test.csv.zip' -d '/content/drive/MyDrive/Health misinformation/zero to ransfer bert/'

Archive:  /content/drive/MyDrive/Health misinformation/zero to ransfer bert/jigsaw-toxic-comment-train.csv.zip
replace /content/drive/MyDrive/Health misinformation/zero to ransfer bert/jigsaw-toxic-comment-train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  /content/drive/MyDrive/Health misinformation/zero to ransfer bert/validation.csv.zip
  inflating: /content/drive/MyDrive/Health misinformation/zero to ransfer bert/validation.csv  
Archive:  /content/drive/MyDrive/Health misinformation/zero to ransfer bert/test.csv.zip
  inflating: /content/drive/MyDrive/Health misinformation/zero to ransfer bert/test.csv  


In [12]:
train = pd.read_csv('/content/drive/MyDrive/Health misinformation/zero to ransfer bert/jigsaw-toxic-comment-train.csv')
validation = pd.read_csv('/content/drive/MyDrive/Health misinformation/zero to ransfer bert/validation.csv')
test = pd.read_csv('/content/drive/MyDrive/Health misinformation/zero to ransfer bert/test.csv')


In [13]:
train.head(3)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0


In [14]:
train.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [15]:
train.drop(['severe_toxic','obscene','threat','insult','identity_hate'],axis=1,inplace=True)

In [17]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223549 entries, 0 to 223548
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            223549 non-null  object
 1   comment_text  223549 non-null  object
 2   toxic         223549 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 5.1+ MB


In [19]:

train=train.loc[:12000,:]
train.shape

(12001, 3)

We will check the maximum number of words that can be present in a comment , this will help us in padding later

This code helps find the longest comment (by word count) in the dataset.

In [20]:
train['comment_text'].apply(lambda x:len(str(x).split())).max()

1403

Writing a function for getting auc score for validation

In [21]:
def roc_auc(predictions,target):
    '''
    This methods returns the AUC Score when given the Predictions
    and Labels
    '''

    fpr, tpr, thresholds = metrics.roc_curve(target, predictions)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc


In [26]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.comment_text.values, train.toxic.values,
                                                  stratify=train.toxic.values,
                                                  random_state=42,
                                                  test_size=0.2, shuffle=True)

# using keras tokenizer here

In [27]:
token = text.Tokenizer(num_words=None)
max_len = 1500

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

#zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [29]:
model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1,
                    output_dim=300,
                    input_shape=(max_len,)))  # Use input_shape instead of input_length
model.add(SimpleRNN(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (None, 1500, 300)           │      13,049,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_2 (SimpleRNN)             │ (None, 100)                 │          40,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,089,301 (49.93 MB)

 Trainable params: 13,089,301 (49.93 MB)

 Non-trainable params: 0 (0.00 B)

In [31]:
model.fit(xtrain_pad, ytrain, epochs=5, batch_size=64 * strategy.num_replicas_in_sync)


Epoch 1/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 366s 2s/step - accuracy: 0.8747 - loss: 0.3512
Epoch 2/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 373s 2s/step - accuracy: 0.9661 - loss: 0.1058
Epoch 3/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 382s 2s/step - accuracy: 0.9959 - loss: 0.0146
Epoch 4/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 383s 2s/step - accuracy: 1.0000 - loss: 0.0017
Epoch 5/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 383s 2s/step - accuracy: 1.0000 - loss: 5.4481e-04
